In [12]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd

In [13]:
def get_url(type, location, transaccion, page):
    return f"https://www.fincaraiz.com.co/{type}/{transaccion}/{location}?pagina={page}"

In [14]:
transaccion = "venta"
ubicacion = "bogota"
tipo = "apartaestudios-apartamentos"

In [15]:
houses = []

main_driver = webdriver.Firefox()

for page in range(30):
    main_url = get_url(tipo, ubicacion, transaccion, page)
    main_driver.get(main_url)
    print(f"Página {page}")
    container = main_driver.find_element(By.ID, "listingContainer")
    articles = main_driver.find_elements(By.TAG_NAME, "article")

    urls = []
    for article in articles:
        url = article.find_element(By.TAG_NAME, "a").get_property("href")
        page_driver = webdriver.Firefox()
        page_driver.get(main_url)
        page_driver.get(url)
        attributes = page_driver.find_elements(By.CLASS_NAME, "jss261")
        house = {}
        #jss1181 jss62 jss1177
        price = page_driver.find_elements(By.CLASS_NAME, "jss62")[0].find_elements(By.TAG_NAME, "p")[1]
        house["price"] = price.text
        for attribute in attributes:
            try:
                name, value = attribute.find_elements(By.TAG_NAME, "p")
                house[name.text] = value.text
            except Exception as e:
                pass
        houses.append(house)
        page_driver.close()

main_driver.close()


Página 0


TimeoutException: Message: 
Stacktrace:
WebDriverError@chrome://marionette/content/error.js:181:5
TimeoutError@chrome://marionette/content/error.js:450:5
bail@chrome://marionette/content/sync.js:227:19


In [ ]:
len(houses)

In [ ]:
houses

In [ ]:
df = pd.DataFrame.from_dict(houses)
df.to_csv(f"house_prices_colombia_{ubicacion}.csv")

In [15]:
df

,price,Habitaciones,Baños,Área construída,Área privada,Estrato,Estado,Antigüedad,Piso N°,Administración,Precio m²,Tipo de apartamento,Parqueaderos
0,$ 130.000.000,3,2,51 m²,0 m²,3,Bueno,1 a 8 años,2,$ 79.000 COP,"$ 2.549.019,61*m²",NaN,NaN
1,$ 112.000.000,2,1,44 m²,44 m²,2,Bueno,9 a 15 años,1,No definida,"$ 2.545.454,55*m²",NaN,NaN
2,$ 150.000.000,3,1,70 m²,64 m²,3,Remodelado,1 a 8 años,6,$ 50 COP,"$ 2.142.857,14*m²",NaN,NaN
3,$ 152.152.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,$ 100.000.000,3,1,56 m²,56 m²,3,Remodelado,9 a 15 años,6,No definida,"$ 1.785.714,29*m²",Loft,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
315,$ 139.000.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
316,$ 120.000.000,2,1,47 m²,42 m²,3,Bueno,16 a 30 años,3,$ 50.000 COP,"$ 2.553.191,49*m²",NaN,NaN
317,$ 137.000.000,3,2,61 m²,61 m²,3,Bueno,1 a 8 años,1,$ 80.000 COP,"$ 2.245.901,64*m²",NaN,NaN
318,$ 149.000.000,3,2,69 m²,70 m²,3,Excelente,1 a 8 años,2,$ 72.000 COP,"$ 2.159.420,29*m²",NaN,NaN
